<a href="https://colab.research.google.com/github/yeoncw1226/2024_2_dataScienceProject_team5/blob/main/DSP_busClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1. 경계 점들로 다각형 생성.
#2. polygon.contains(point)

#def(경계, 점):
  #return T/F

In [ ]:
import json

# Load and inspect the GeoJSON file
with open('/content/drive/MyDrive/서울_자치구_경계_2017.geojson', 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

# Print the first feature to inspect its structure
print(geojson_data['features'][0])


{'type': 'Feature', 'properties': {'SIG_CD': '11110', 'SIG_ENG_NM': 'Jongno-gu', 'SIG_KOR_NM': '종로구'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[127.00864326221884, 37.58046825204727], [127.00871274905404, 37.58045116513171], [127.00876564011087, 37.580443107078715], [127.00890785297045, 37.580424231608816], [127.00913781377908, 37.580393529395884], [127.00916523299747, 37.58039071654565], [127.00923792440939, 37.58038254887855], [127.00926870590294, 37.580379168163], [127.0092779097711, 37.580378032315615], [127.0094381678049, 37.5803566322574], [127.01003197993037, 37.58027803392712], [127.01003587429, 37.58027746602481], [127.01029590502878, 37.58026618464937], [127.01059113253233, 37.580253773768696], [127.01067835517416, 37.58039257300021], [127.01070383209944, 37.58043030433113], [127.01097470383016, 37.5808272845678], [127.0111144791085, 37.581044068617175], [127.01117197391697, 37.581091924322095], [127.01160983077278, 37.58143538872224], [127.01169173781155, 37.5815063

In [ ]:
import json
from shapely.geometry import Point, Polygon, MultiPolygon

# 파일 로드
with open('/content/drive/MyDrive/서울_자치구_경계_2017.geojson', 'r', encoding='utf-8') as file:
    geojson_data = json.load(file)

# 전역 변수: 구별 폴리곤 저장
polygons_by_sig_kor_nm = {}

# GeoJSON 파일 처리 함수
def process_geojson(geojson_data):
    polygons = {}
    for feature in geojson_data['features']:
        sig_kor_nm = feature["properties"]["SIG_KOR_NM"]
        geometry = feature["geometry"]

        if geometry["type"] == "Polygon":
            coordinates = geometry["coordinates"][0]
            polygon = Polygon(coordinates)
        elif geometry["type"] == "MultiPolygon":
            coordinates = [Polygon(poly[0]) for poly in geometry["coordinates"]]
            polygon = MultiPolygon(coordinates)
        else:
            print(f"Unsupported geometry type: {geometry['type']}")
            continue

        polygons[sig_kor_nm] = polygon

    print("Polygons generated successfully!")
    return polygons

# GeoJSON 데이터로부터 폴리곤 생성
polygons_by_sig_kor_nm = process_geojson(geojson_data)
print(f"Loaded {len(polygons_by_sig_kor_nm)} polygons.")
print(f"Available regions: {list(polygons_by_sig_kor_nm.keys())}")

# 특정 점이 속한 구 찾기 함수
def find_region_for_point(lat, lon):
    point = Point(lon, lat)  # Shapely는 (x, y) 순서로 받음
    for sig_kor_nm, polygon in polygons_by_sig_kor_nm.items():
        if polygon.contains(point):
            return sig_kor_nm
    return None

# 테스트 함수 예제
def test_point_in_region(lat, lon):
    result = find_region_for_point(lat, lon)
    if result:
        print(f"The point ({lat}, {lon}) is in {result}.")
    else:
        print(f"The point ({lat}, {lon}) is not in any region.")

Polygons generated successfully!
Loaded 25 polygons.
Available regions: ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']


In [ ]:
import pandas as pd

# CSV 파일 경로
bus_stop_file_path = '/content/drive/MyDrive/서울버정.csv'

# CSV 파일 읽기
bus_stops = pd.read_csv(bus_stop_file_path, encoding='euc-kr')

# 결과 저장: 정류소 이름, 위도, 경도, 구 이름
def assign_regions_to_stops(df, polygons):
    results = []
    for _, row in df.iterrows():
        lat, lon = row['위도'], row['경도']
        region = find_region_for_point(lat, lon)  # find_region_for_point 함수 사용
        results.append(region)
    df['구 이름'] = results
    return df

# 정류소 데이터에 '구 이름' 추가
bus_stops_with_regions = assign_regions_to_stops(bus_stops, polygons_by_sig_kor_nm)

# 결과 출력 확인
print(bus_stops_with_regions.head())

# 필요시 다른 로직에서 bus_stops_with_regions 사용


            정류소명          경도         위도 구 이름
0        종로2가사거리  126.987752  37.569806  종로구
1    창경궁.서울대학교병원  126.996521  37.579433  종로구
2      명륜3가.성대입구  126.998251  37.582580  종로구
3       종로2가.삼일교  126.987613  37.568579  종로구
4  혜화동로터리.여운형활동터  127.001744  37.586243  종로구


In [ ]:
import pandas as pd

# CSV 파일 경로
bus_stop_file_path = '/content/drive/MyDrive/서울버정.csv'

# CSV 파일 읽기 (인코딩 지정)
bus_stops = pd.read_csv(bus_stop_file_path, encoding='euc-kr')

# 결과 저장: 정류소 이름, 위도, 경도, 구 이름
def assign_regions_to_stops(df, polygons):
    results = []
    for _, row in df.iterrows():
        lat, lon = row['위도'], row['경도']
        region = find_region_for_point(lat, lon)  # find_region_for_point 함수 사용
        results.append(region)
    df['구 이름'] = results
    return df

# 정류소 데이터에 '구 이름' 추가
bus_stops_with_regions = assign_regions_to_stops(bus_stops, polygons_by_sig_kor_nm)

# 수정된 데이터 저장
output_file_path = '/content/drive/MyDrive/서울버정_with_regions.csv'
bus_stops_with_regions.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"구 이름 레이블이 추가된 데이터를 다음 경로에 저장했습니다: {output_file_path}")


구 이름 레이블이 추가된 데이터를 다음 경로에 저장했습니다: /content/drive/MyDrive/서울버정_with_regions.csv


In [ ]:
import pandas as pd

# CSV 파일 경로
bus_stop_file_path = '/content/drive/MyDrive/seoulsubway.csv'

# CSV 파일 읽기 (인코딩 지정)
bus_stops = pd.read_csv(bus_stop_file_path, encoding='euc-kr')

# 결과 저장: 정류소 이름, 위도, 경도, 구 이름
def assign_regions_to_stops(df, polygons):
    results = []
    for _, row in df.iterrows():
        lat, lon = row['위도'], row['경도']
        region = find_region_for_point(lat, lon)  # find_region_for_point 함수 사용
        results.append(region)
    df['구 이름'] = results
    return df

# 정류소 데이터에 '구 이름' 추가
bus_stops_with_regions = assign_regions_to_stops(bus_stops, polygons_by_sig_kor_nm)

# 수정된 데이터 저장
output_file_path = '/content/drive/MyDrive/지하철_with_regions.csv'
bus_stops_with_regions.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"구 이름 레이블이 추가된 데이터를 다음 경로에 저장했습니다: {output_file_path}")

구 이름 레이블이 추가된 데이터를 다음 경로에 저장했습니다: /content/drive/MyDrive/지하철_with_regions.csv


In [ ]:
import pandas as pd

# CSV 파일 경로
file_path = '/content/drive/MyDrive/서울지하철_자치구추가 (1).csv'

# CSV 파일 읽기 (인코딩 지정)
bus_stops = pd.read_csv(file_path, encoding='utf-8')

bus_stops['역사명'].drop_duplicates()
bus_stops = bus_stops[bus_stops['구 이름'] == '광진구']

# 수정된 데이터 저장
output_file_path = '/content/drive/MyDrive/서울지하철_자치구추가 (1).csv'
bus_stops.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"구 이름 레이블이 추가된 데이터를 다음 경로에 저장했습니다: {output_file_path}")

구 이름 레이블이 추가된 데이터를 다음 경로에 저장했습니다: /content/drive/MyDrive/서울지하철_자치구추가 (1).csv
